In [1]:
#import libraries
import re
import requests
import pandas as pd

In [44]:
#extraction make mode year
def extract_make_model_year(title):
    # match = re.match(r"(?P<make>\w+)\s+(?P<model>[A-Za-z0-9\-]+)?\s+(?P<year>\d{4})?", title.strip())
    pattern = r"(?P<make>\b[a-zA-Z\-]+)\s+(?P<model>[A-Za-z0-9\-]+(?:\s[A-Za-z0-9\-]+)?)?.*?(?P<year>\d{4})(?!\d)"
    match = re.search(pattern, title.strip())
    if match:
        return match.group("make"), match.group("model"), match.group("year")
    return None, None, None

In [45]:
# extraction condition
def extract_condition(condition):
    condition_lower = condition.lower()
    if "foreign used" in condition_lower:
        return "foreign used"
    elif "local used" in condition_lower:
        return "local used"
    elif "new" in condition_lower:
        return "new"
    else:
        return None

In [46]:
#extraction transmission
def extract_transmission(transmission):
    transmission_lower = transmission.lower()
    if "automatic" in transmission_lower:
        return "automatic"
    elif "manual" in transmission_lower:
        return "manual"
    else:
        return None

In [47]:
#fetch the jiji api
def fetch_json_data(page):
    url = "https://jiji.ng/api_web/v1/listing"
    params = {
        "slug": "cars",
        "page": page,
        "webp": True
    }
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException:
        print(f"[Page] {page}. Request Error")
        return []
    except ValueError:
        print(f"[Page] {page}. Decode json Error")
        return []

    adverts = data.get("adverts_list", {}).get("adverts", [])
    if not isinstance(adverts, list):
        print(f"An error occured. Expected list but got {type(adverts)}")
        return []
    return adverts

In [48]:
def get_attrs_value(attrs, key_name):
    for attr in attrs:
        if attr.get("name", "").lower() == key_name.lower():
            return attr.get("value", "").strip()
    return None

In [51]:
def main():
    all_ads = []
    for page in range(1, 101):
        ads = fetch_json_data(page)
        print(f"Page {page}: {len(ads)} found.")

        for ad in ads:
            if isinstance(ad, dict):
                attrs = ad.get("attrs", [])
                title = ad.get("title", "")
                condition_ = get_attrs_value(attrs, "condition")
                transmission_ = get_attrs_value(attrs, "transmission")
                condition = extract_condition(condition_)
                transmission = extract_transmission(transmission_)
                make, model, year = extract_make_model_year(title)
                price = ad.get("price_title", "")

                if price:
                    all_ads.append({
                        "title": title,
                        "make": make,
                        "model": model,
                        "year": year,
                        "condition": condition,
                        "transmission": transmission,
                        "price": price
                    })
    if all_ads:
        df = pd.DataFrame(all_ads)
        df.to_csv("Jiji_car_dataset.csv", index=False)
        # df.to_csv("car_dataset.csv", index=False)
        print("Successfully extract jiji car data")
    else:
        print("No extraction")
                
        

In [52]:
if __name__ == "__main__":
    main()

Page 1: 20 found.
Page 2: 20 found.
Page 3: 20 found.
Page 4: 20 found.
Page 5: 20 found.
Page 6: 20 found.
Page 7: 20 found.
Page 8: 20 found.
Page 9: 20 found.
Page 10: 20 found.
Page 11: 20 found.
Page 12: 20 found.
Page 13: 20 found.
Page 14: 20 found.
Page 15: 20 found.
Page 16: 20 found.
Page 17: 20 found.
Page 18: 20 found.
Page 19: 20 found.
Page 20: 20 found.
Page 21: 20 found.
Page 22: 20 found.
Page 23: 20 found.
Page 24: 20 found.
Page 25: 20 found.
Page 26: 20 found.
Page 27: 20 found.
Page 28: 20 found.
Page 29: 20 found.
Page 30: 20 found.
Page 31: 20 found.
Page 32: 20 found.
Page 33: 20 found.
Page 34: 20 found.
Page 35: 20 found.
Page 36: 20 found.
Page 37: 20 found.
Page 38: 20 found.
Page 39: 20 found.
Page 40: 20 found.
Page 41: 20 found.
Page 42: 20 found.
Page 43: 20 found.
Page 44: 20 found.
Page 45: 20 found.
Page 46: 20 found.
Page 47: 20 found.
Page 48: 20 found.
Page 49: 20 found.
Page 50: 20 found.
Page 51: 20 found.
Page 52: 20 found.
Page 53: 20 found.
Pa